# Cálculo de la pérdida y precisión de la clasificación

Durante esta sección, se ha preparado un conjunto de datos, cargado un modelo preentrenado y modificado para el ajuste fino de la clasificación. ANtes de proceder con el ajuste fino, solo queda: implementar  las  funciones  de  evaluación  del  modelo  utilizadas  durante  el  ajuste  fino.

![Texto alternativo](./imgs/6.12.png)

Antes  de  implementar  las  utilidades  de  evaluación, hay que analizar  brevemente  cómo  se convierten  las  salidas  del  modelo  en  predicciones  de  etiquetas  de  clase.

En  la sección 05,  se calculaba el  ID  del  siguiente  token  generado  por  el  LLM  convirtiendo  las  50 257  salidas  en  probabilidades  mediante  la  función  softmax  y  devolviendo  la  posición  de  mayor  probabilidad  mediante  la  función  argmax . En esta sección,  se utiliza el  mismo  enfoque  para  calcular  si  el  modelo  genera  una  predicción  de  "spam"  o  "no  spam"  para  una  entrada  dada,  con  la  única  diferencia  de  que  se trabaja  con  salidas  bidimensionales  en  lugar  de  50 257  dimensiones

![Texto alternativo](./imgs/6.13.png)

In [3]:
import sys
import os

# Obtiene la ruta de la carpeta principal del proyecto (subiendo un nivel desde seccion05)
ruta_proyecto_principal = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añade esta ruta a la lista de lugares donde Python busca módulos
if ruta_proyecto_principal not in sys.path:
    sys.path.append(ruta_proyecto_principal)


In [5]:
import tiktoken
import torch
from gptModelClasification import return_clasification_model
model = return_clasification_model()
tokenizer = tiktoken.get_encoding("gpt2")
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
with torch.no_grad():
    outputs = model(inputs)
print(outputs[:,-1,:])

File already exists and is up-to-date: gpt2\124M\checkpoint
File already exists and is up-to-date: gpt2\124M\encoder.json
File already exists and is up-to-date: gpt2\124M\hparams.json
File already exists and is up-to-date: gpt2\124M\model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2\124M\model.ckpt.index
File already exists and is up-to-date: gpt2\124M\model.ckpt.meta
File already exists and is up-to-date: gpt2\124M\vocab.bpe
tensor([[-3.5983,  3.9902]])


UNa vez obtenido los valores del tensor correspondiente al último token se puede obtener la etiqueta a traves de:

In [6]:
probas = torch.softmax(outputs[:,-1,:], dim=-1)
label = torch.argmax(probas)
print("Class label:", label.item())

Class label: 1


En  este  caso,  el  código  devuelve  1,  lo  que  significa  que  el  modelo  predice  que  el  texto  de  entrada  es  "spam".
El  uso  de  la  función  softmax  aquí  es  opcional  porque  los  resultados  más  grandes  corresponden  directamente  a  los  puntajes  de  probabilidad  más  altos.  Por  lo  tanto,  se puede simplificar  el  código  de  la  siguiente  manera,  sin  usar  softmax:

In [16]:
logits = outputs[:, -1, :]
label = torch.argmax(logits)
print("Class label:", label.item())


Class label: 1


Este  concepto  se  puede  utilizar  para  calcular  la  denominada  precisión  de  clasificación,  que  mide  el  porcentaje  de  predicciones  correctas  en  un  conjunto  de  datos.

Para  determinar  la  precisión  de  la  clasificación,  se aplica  el  código  de  predicción  basado  en  argmax  a  todos  los  ejemplos  del  conjunto  de  datos  y  se calcula  la  proporción  de  predicciones  correctas  definiendo  una  función  calc_accuracy_loader :

In [19]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)
            with torch.no_grad():
                logits = model(input_batch)[:, -1, :]             #logits del último token de cada batch
            predicted_labels = torch.argmax(logits, dim=-1)
            num_examples += predicted_labels.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break
        return correct_predictions / num_examples

In [39]:
from spamDataset import SpamDataset
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 10
train_dataset = SpamDataset(csv_file="CSV/train.csv",max_length=None,tokenizer=tokenizer)
val_dataset = SpamDataset(csv_file="CSV/validation.csv",max_length=train_dataset.max_length,tokenizer=tokenizer)
test_dataset = SpamDataset(csv_file="CSV/test.csv",max_length=train_dataset.max_length,tokenizer=tokenizer)

torch.manual_seed(123)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=num_workers,drop_last=True,)
val_loader = DataLoader(dataset=val_dataset,batch_size=batch_size,num_workers=num_workers,drop_last=False,)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,num_workers=num_workers,drop_last=False,)

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.manual_seed(123)
train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=10)
val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=10)
test_accuracy = calc_accuracy_loader(test_loader, model, device, num_batches=10)
print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")

Training accuracy: 30.00%
Validation accuracy: 50.00%
Test accuracy: 60.00%


Para mejorar el modelo es necesario realizar unos ajustes sobre el.
Sin  embargo,  antes  de  comenzar  a  ajustar  el  modelo, se necesita definir  la  función  de  pérdida  que se optimizara durante  el  entrenamiento. El  objetivo  es  maximizar  la  precisión  de  la  clasificación  de  spam  del  modelo,  lo  que  significa  que  el  código  anterior  debe  generar  las  etiquetas  de  clase  correctas:  0  para  textos  que  no  son  spam  y  1  para  textos  spam.
Sin  embargo,  la  precisión  de  la  clasificación  no  es  una  función  diferenciable,  por  lo  que  se utiliza  la  pérdida  de  entropía  cruzada  como  indicador  para  maximizar  la  precisión.  Esta  es  la  misma  pérdida  de  entropía  cruzada  que  se  analizó  en  la sección 05.
En  consecuencia,  la  función  calc_loss_batch  sigue  siendo  la  misma  que  en  la sección  05,  con  un  ajuste:  hay que centrarse  en  optimizar  solo  el  último  token,  model(input_batch)[:,  1, :],  en  lugar  de  todos  los  tokens,  model(input_batch):

In [41]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)[:,-1,:]
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss

Calcular  la  pérdida  de  un  único  lote  obtenido  de  los  cargadores  de  datos  previamente  definidos.   Para  calcular  la  pérdida  de  todos  los  lotes  de  un  cargador  de  datos, se define  la  función  calc_loss_loader :

In [42]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:                                                      
        num_batches = min(num_batches, len(data_loader))

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [49]:
with torch.no_grad():                                      
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=5)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)
    test_loss = calc_loss_loader(test_loader, model, device, num_batches=5)
print(f"Training loss: {train_loss:.3f}")
print(f"Validation loss: {val_loss:.3f}")
print(f"Test loss: {test_loss:.3f}")

Training loss: 2.474
Validation loss: 2.688
Test loss: 2.365


En la siguiente sección se implementará  una  función  de  entrenamiento  para  ajustar  el  modelo,  lo  que  implica  ajustarlo  para  minimizar  la  pérdida  del  conjunto  de  entrenamiento.  Minimizar  la  pérdida  del  conjunto  de  entrenamiento  ayudará  a  aumentar  la  precisión  de  la  clasificación,  al  objetivo  general.


[Ajuste del modelo en datos supervisados](./7_ajuste_modelo_datos_supervisados.ipynb)